In [1]:
from model import DFL_VGG16,DFL_RESNET,DFL_EfficientNet,MangoNet,MangoRegressionNet
from dataset import MangoDataset
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from util import mixup_data, mixup_criterion, plot_confusion_matrix
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import torch
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
%matplotlib inline

## load data

In [3]:
BATCH_SIZE=8
trainset = MangoDataset(root = './data/', train = "train")
valset = MangoDataset(root = './data/', train = "val")
testset = MangoDataset(root = './data/', train = "test")
# create train/val loaders
train_loader = DataLoader(dataset=trainset,
                          batch_size=BATCH_SIZE, 
                          shuffle=True,
                          num_workers=1)

val_loader = DataLoader(dataset=valset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False,
                        num_workers=1)

test_loader = DataLoader(dataset=testset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False,
                        num_workers=1)

train_n = len(trainset)
val_n = len(valset)


/home/deep307-2-4/Data/chekuei/mango/dataset.py:42: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  label = np.where(label=="A", 0, label)
/home/deep307-2-4/Data/chekuei/mango/dataset.py:43: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  label = np.where(label=="B", 1, label)
/home/deep307-2-4/Data/chekuei/mango/dataset.py:44: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  numerical_lbls = np.where(label=="C", 2, label)


In [4]:
EPOCH = 50
DEVICE = "cuda:1"
model_path = "model/"

## DFL 

In [ ]:

# model = DFL_EfficientNet().to(DEVICE)
# model = DFL_RESNET().to(DEVICE)
model = DFL_VGG16().to(DEVICE)
model.load_state_dict(torch.load(model_path+"DFL_VGG16"))
ce = torch.nn.NLLLoss()
# opt = torch.optim.Adam([{'params': model.parameters(), 'lr': 0.0001}]) ## for efficientnet
opt = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.000005)
for i in range(EPOCH):
    model.train()
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1, out2, out3 = model(data)
        out1, out2, out3 = F.log_softmax(out1,1), F.log_softmax(out2,1), F.log_softmax(out3,1)
        loss = ce(out1,label)+ce(out2,label)+0.1*ce(out3,label)
        loss.backward()
        opt.step()
        opt.zero_grad()
    print(loss)

    model.eval()
    correct = 0
    for data, label in tqdm(val_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1, out2, out3 = model(data)
        out = out1+out2+0.1*out3
        pred = torch.max(out,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("valacc:",correct/val_n)
    model.eval()
    correct = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1, out2, out3 = model(data)
        out = out1+out2+0.1*out3
        pred = torch.max(out,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("trainacc:",correct/train_n)
#     torch.save(model.state_dict(), "DFL_VGG16_1")

## EfficientNet + Mixup

In [ ]:
model = MangoNet().to(DEVICE)

ce = torch.nn.NLLLoss()
opt = torch.optim.Adam([{'params': model.parameters(), 'lr': 0.0001}])
# opt = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.000005)
for i in range(EPOCH):
    model.train()
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)
        out1 = F.log_softmax(out1,1)
        loss = ce(out1,label)
        mixed_x, y_a, y_b, lam = mixup_data(data, label)
        pred = model(mixed_x)
        mixloss = mixup_criterion(ce, pred, y_a, y_b, lam)
        closs = loss+mixloss
        closs.backward()
        opt.step()
        opt.zero_grad()
    print(closs)
    model.eval()
    correct = 0
    for data, label in tqdm(val_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)

        pred = torch.max(out1,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("valacc:",correct/val_n)
    model.eval()
    correct = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)
        pred = torch.max(out1,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("trainacc:",correct/train_n)
    torch.save(model.state_dict(), model_path+"model")
#     

## DFL Feature Visualization

In [ ]:
model = DFL_VGG16().to(DEVICE)
model.load_state_dict(torch.load(model_path+"DFL_VGG16"))
model.eval()
correct = 0
preds,labels,val_label,val_pred=[],[],[],[]
test_feature_a,train_feature_a,val_feature_a=[],[],[]
for data in tqdm(test_loader):
    data = data.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    test_feature_a.append(feature_b.data.cpu())
    out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    val_feature_a.append(feature_b.data.cpu())
    val_label.append(label.cpu())
for data,label in tqdm(train_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    train_feature_a.append(feature_b.data.cpu())
    labels.append(label.cpu())
test_feature_a = np.asarray(torch.cat(test_feature_a).cpu().numpy())
train_feature_a = np.asarray(torch.cat(train_feature_a).cpu().numpy())
val_feature_a = np.asarray(torch.cat(val_feature_a).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
val_label = np.asarray(torch.cat(val_label).cpu().numpy())
n=300
a = np.concatenate((train_feature_a[:n].reshape(n,-1),test_feature_a[:n].reshape(n,-1)),axis=0)
a = np.concatenate((a,val_feature_a[:n].reshape(n,-1)),axis=0)
X_embedded = TSNE(n_components=2).fit_transform(a)
plt.scatter(X_embedded[:n,0],X_embedded[:n,1],c=labels[:n],label="train",marker=".",s=50)
plt.scatter(X_embedded[n:2*n,0],X_embedded[n:2*n,1],c="black",label="test",marker=".",s=50)
plt.scatter(X_embedded[2*n:,0],X_embedded[2*n:,1],c=val_label[:n],label="val",marker=".",s=50)
plt.legend(loc='upper right')
plt.show()

## EfficientNet Feature Visualization

In [ ]:

preds,labels,val_label,val_pred=[],[],[],[]
test_feature_a,train_feature_a,val_feature_a=[],[],[]
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load(model_path+"EfficientNet"))
for data in tqdm(test_loader):
    data = data.to(DEVICE)
    feature_b = model.backbone.extract_features(data)
    test_feature_a.append(feature_b.data.cpu())
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    feature_b = model.backbone.extract_features(data)
    val_feature_a.append(feature_b.data.cpu())
    val_label.append(label.cpu())
for data,label in tqdm(train_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    feature_b = model.backbone.extract_features(data)
    train_feature_a.append(feature_b.data.cpu())
    labels.append(label.cpu())
test_feature_a = np.asarray(torch.cat(test_feature_a).cpu().numpy())
train_feature_a = np.asarray(torch.cat(train_feature_a).cpu().numpy())
val_feature_a = np.asarray(torch.cat(val_feature_a).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
val_label = np.asarray(torch.cat(val_label).cpu().numpy())

train_feature_a = np.mean(train_feature_a,axis=(2,3))
test_feature_a = np.mean(test_feature_a,axis=(2,3))
val_feature_a = np.mean(val_feature_a,axis=(2,3))
n=300
a = np.concatenate((train_feature_a[:n].reshape(n,-1),test_feature_a[:n].reshape(n,-1)),axis=0)
a = np.concatenate((a,val_feature_a[:n].reshape(n,-1)),axis=0)
X_embedded = TSNE(n_components=2).fit_transform(a)
plt.scatter(X_embedded[:n,0],X_embedded[:n,1],c=labels[:n],label="train",marker=".",s=50)
plt.scatter(X_embedded[n:2*n,0],X_embedded[n:2*n,1],c="black",label="test",marker=".",s=50)
plt.scatter(X_embedded[2*n:,0],X_embedded[2*n:,1],c=val_label[:n],label="val",marker=".",s=50)
plt.legend(loc='upper right')
# plt.savefig(method+"scatter"+str(step)+".png")
plt.show()

## EfficientNet Testing

In [ ]:
DEVICE = "cuda:1"
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load(model_path+"EfficientNet"))
model.eval()
correct = 0
preds=[]
labels=[]
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out = model(data)
#     out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
    labels.append(label)
preds = np.asarray(torch.cat(preds).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())

## Confusion Matrix

In [ ]:
model = DFL_VGG16().to(DEVICE)
model.load_state_dict(torch.load(model_path+"DFL_VGG16"))
model.eval()
correct = 0
preds=[]
labels=[]
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
    labels.append(label)
preds = np.asarray(torch.cat(preds).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
cnf_matrix = confusion_matrix(labels, preds)
plot_confusion_matrix(cnf_matrix,classes=[a for a in range(3)],normalize=True)
plt.show()

DEVICE = "cuda:0"
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load(model_path+"EfficientNet"))
model.eval()
correct = 0
preds,labels=[],[]
for data,label in tqdm(val_loader):
    data = data.to(DEVICE)
    out = model(data)
#     out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    labels.append(label)
    preds.append(pred)
preds = np.asarray(torch.cat(preds).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
cnf_matrix = confusion_matrix(labels, preds)
plot_confusion_matrix(cnf_matrix,classes=[a for a in range(3)],normalize=True)
plt.show()

## Submit Result

In [ ]:

DEVICE = "cuda:1"
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load(model_path+"EfficientNet"))
model.eval()
preds=[]
for data in tqdm(test_loader):
    data = data.to(DEVICE)
    out = model(data)
#     out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
preds = np.asarray(torch.cat(preds).cpu().numpy())
preds = np.where(preds==0, "A", preds)
preds = np.where(preds=="1", "B", preds)
preds = np.where(preds=="2", "C", preds)
df = pd.read_csv("test_example.csv")
df["label"]=pd.Series(preds)
df = df.set_index("image_id")
df.to_csv("result_DFL.csv")

## Efficient Regression

In [9]:

EPOCH = 50
DEVICE = "cuda:1"
model = MangoRegressionNet().to(DEVICE)

ce = torch.nn.NLLLoss()
mse = torch.nn.MSELoss()
opt = torch.optim.Adam([{'params': model.parameters(), 'lr': 0.0001}])
# opt = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.000005)
for i in range(EPOCH):
#     model.train()
#     for data, label in tqdm(train_loader):
#         data, label = data.to(DEVICE), label.to(DEVICE).float()/2
#         out1 = model(data)
#         out1 = F.sigmoid(out1)
#         loss = mse(out1,label)
# #         mixed_x, y_a, y_b, lam = mixup_data(data, label)
# #         pred = model(mixed_x)
# #         mixloss = mixup_criterion(ce, pred, y_a, y_b, lam)
#         closs = loss#+mixloss
#         closs.backward()
#         opt.step()
#         opt.zero_grad()
#     print(closs)
    model.eval()
    correct = 0
    for data, label in tqdm(val_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)
        out1 = F.sigmoid(out1)
        print(out1)
        out1[out1< 0.33]=0
        out1[out1>0.66]=2
        out1[(0.33< out1) & (out1 < 0.66)]=1
        
        print(label)
        print(out1)
        correct += torch.sum((label==out1.reshape(-1)).int()).detach().item()
    print("valacc:",correct/val_n)
    model.eval()
    correct = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)
        out1 = F.sigmoid(out1)
        out1 = torch.where(out1 < 0.33, torch.tensor(0.), out1)
        out1 = torch.where((0.33< out1) & (out1 < 0.66), torch.tensor(1.), out1)
        pred = torch.where(out1 > 0.66, torch.tensor(2.), out1)
        correct += torch.sum((label==pred).int()).detach().item()
    print("trainacc:",correct/train_n)
    torch.save(model.state_dict(), model_path+"regressionmodel")

  0%|          | 0/100 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b7


  2%|▏         | 2/100 [00:00<00:22,  4.39it/s]

tensor([[0.5178],
        [0.5179],
        [0.5422],
        [0.5392],
        [0.5222],
        [0.4643],
        [0.5047],
        [0.5571]], device='cuda:1', grad_fn=<SigmoidBackward>)
tensor([1, 0, 1, 1, 2, 1, 2, 0], device='cuda:1')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:1', grad_fn=<IndexPutBackward>)
tensor([[0.5461],
        [0.5279],
        [0.5919],
        [0.5314],
        [0.5306],
        [0.5181],
        [0.5011],
        [0.5429]], device='cuda:1', grad_fn=<SigmoidBackward>)
tensor([1, 0, 0, 1, 2, 1, 2, 1], device='cuda:1')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:1', grad_fn=<IndexPutBackward>)


  4%|▍         | 4/100 [00:00<00:16,  5.91it/s]

...

tensor([[0.5824],
        [0.5238],
        [0.5482],
        [0.5315],
        [0.5049],
        [0.4984],
        [0.5180],
        [0.5163]], device='cuda:1', grad_fn=<SigmoidBackward>)
tensor([2, 1, 0, 0, 0, 1, 0, 2], device='cuda:1')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:1', grad_fn=<IndexPutBackward>)
tensor([[0.5347],
        [0.5730],
        [0.5080],
        [0.5143],
        [0.5573],
        [0.5621],
        [0.5231],
        [0.5577]], device='cuda:1', grad_fn=<SigmoidBackward>)
tensor([1, 1, 0, 2, 0, 1, 2, 0], device='cuda:1')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:1', grad_fn=<IndexPutBackward>)


 20%|██        | 20/100 [00:02<00:09,  8.17it/s]

tensor([[0.4941],
        [0.5589],
        [0.5081],
        [0.5678],
        [0.5064],
        [0.4925],
        [0.4821],
        [0.4952]], device='cuda:1', grad_fn=<SigmoidBackward>)
tensor([0, 0, 2, 1, 1, 1, 1, 1], device='cuda:1')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:1', grad_fn=<IndexPutBackward>)
tensor([[0.5235],
        [0.5015],
        [0.5269],
        [0.5316],
        [0.5149],
        [0.5334],
        [0.5532],
        [0.5131]], device='cuda:1', grad_fn=<SigmoidBackward>)
tensor([1, 1, 1, 1, 1, 0, 2, 1], device='cuda:1')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:1', grad_fn=<IndexPutBackward>)


 20%|██        | 20/100 [00:02<00:10,  7.48it/s]


KeyboardInterrupt: 